# [ICA][HW3] Support Vector Machine (SVM)

Implementacao manual de SVM (margem suave) com gradiente descendente, seguindo o mesmo pipeline de pre-processamento dos outros notebooks.

In [10]:
import kagglehub
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer


In [11]:
# 1. Carregar Dados do Kaggle
def carregarDados(caminhoData: str) -> pd.DataFrame:
    """
    Carrega e unifica os arquivos CSV do dataset de HRV via 'uuid'.
    """

    dataframesBrutos = {}
    try:
        arquivosCsv = [f for f in os.listdir(caminhoData) if f.endswith('.csv')]
        for arquivo in arquivosCsv:
            nomeChave = arquivo.replace('.csv', '')
            dataframesBrutos[nomeChave] = pd.read_csv(os.path.join(caminhoData, arquivo))

        listaChaves = list(dataframesBrutos.keys())
        dfUnificado = dataframesBrutos[listaChaves[0]]
        for chave in listaChaves[1:]:
            dfUnificado = pd.merge(dfUnificado, dataframesBrutos[chave], on='uuid', how='inner')

        print(f"Dataset carregado! E unificado:\n    {dfUnificado.shape}")
        return dfUnificado
    except Exception as e:
        print(f"Erro no carregamento: {e}")
        return None



# 2. Selecao e Filtragem dos Dados
def selecionarPreditoresETarget(dfEntrada: pd.DataFrame):
    """
    Isola os preditores, remove classes intermediarias e elimina variaveis
    com multicolinearidade extrema (correlacao > 0.95).
    """

    # 1. Filtragem: Focar nos extremos (No Stress vs Time Pressure)
    # Removemos 'interruption' para limpar a fronteira de decisao e
    # realizar uma classificacao binaria, apenas
    dfLimpo = dfEntrada[dfEntrada['condition'] != 'interruption'].copy()
    dfLimpo = dfLimpo.reset_index(drop=True)

    # 2. Mapeamento Binario
    mapeamentoClasses = {'no stress': 0, 'time pressure': 1}
    y = dfLimpo['condition'].map(mapeamentoClasses).values

    # 3. Identificacao Automatica de Preditores Numericos
    # Removemos colunas nao-preditoras e o proprio alvo
    colunasExcluir = ['uuid', 'datasetId', 'condition', 'target']
    xBruto = dfLimpo.drop(columns=[c for c in colunasExcluir if c in dfLimpo.columns])
    xBruto = xBruto.select_dtypes(include=[np.number])

    # 4. Filtro de Multicolinearidade (Estrategia Inteligente)
    # Calculamos a matriz de correlacao absoluta
    matrizCorr = xBruto.corr().abs()

    # Selecionamos o triangulo superior da matriz para identificar pares
    superior = matrizCorr.where(np.triu(np.ones(matrizCorr.shape), k=1).astype(bool))

    # Identificamos colunas com correlacao acima de 0.95
    colunasParaRemover = [coluna for coluna in superior.columns if any(superior[coluna] > 0.95)]

    x = xBruto.drop(columns=colunasParaRemover)

    print(f"Filtragem concluida.\n  Preditores finais: {x.shape[1]} (Removidas {len(colunasParaRemover)} redundantes).")

    return x, y

# 2.1 Equilibrio de Amostras (Opcional)
def aplicarSubamostragem(xDados, yAlvo):
    """
    Realiza o Undersampling aleatorio da classe majoritaria para equilibrar o dataset.
    Retorna X e Y balanceados (proporcao 50/50).
    """

    # Identificamos os indices de cada classe
    indicesClasse0 = np.where(yAlvo == 0)[0]
    indicesClasse1 = np.where(yAlvo == 1)[0]

    # Determinamos o tamanho da menor classe (geralmente Estresse)
    n_minoria = len(indicesClasse1)

    # Sorteamos aleatoriamente a mesma quantidade na classe majoritaria
    np.random.seed(27)
    indicesClasse0_Reduzidos = np.random.choice(indicesClasse0, n_minoria, replace=False)

    # Combinamos os indices e extraimos os dados
    indicesFinais = np.concatenate([indicesClasse0_Reduzidos, indicesClasse1])
    np.random.shuffle(indicesFinais)

    xBalanceado = xDados.iloc[indicesFinais] if isinstance(xDados, pd.DataFrame) else xDados[indicesFinais]
    yBalanceado = yAlvo[indicesFinais]

    print(f"Subamostragem concluida: {len(yBalanceado)} amostras totais (50/50).")

    return xBalanceado, yBalanceado


# 3. Divisao dos Conjuntos de Dados
def dividirDados(xDados, yAlvo, proporcaoTeste=0.2):
    """
    Apenas separa os dados em conjuntos de treino e teste de forma estratificada.
    """

    xTreino, xTeste, yTreino, yTeste = train_test_split(
        xDados, yAlvo, test_size=proporcaoTeste, random_state=42, stratify=yAlvo
    )
    print(f"Divisao dos conjuntos concluida (Teste: {proporcaoTeste*100}%).")
    return xTreino, xTeste, yTreino, yTeste


# 4. Transformacao e normalizacao dos dados
def aplicarTransformacoes(xTreino, xTeste):
    """
    Aplica transformacoes para aproximar a distribuicao normal e padroniza as escalas.
    Utiliza Yeo-Johnson para corrigir assimetria e StandardScaler para Z-score.
    """

    # 1. Transformacao de Potencia (Para aproximar da normalidade multivariada)
    powerTrans = PowerTransformer(method='yeo-johnson')

    # 2. Padronizacao (Media 0, Variancia 1)
    scaler = StandardScaler()

    # Aplicamos o pipeline de transformacao
    # Nota: fit() apenas no treino para evitar vazamento de informacao!
    xTreinoTrans = powerTrans.fit_transform(xTreino)
    xTreinoFinal = scaler.fit_transform(xTreinoTrans)

    xTesteTrans = powerTrans.transform(xTeste)
    xTesteFinal = scaler.transform(xTesteTrans)

    print("Transformacao Yeo-Johnson e padronizacao z-score aplicadas.")
    return xTreinoFinal, xTesteFinal


In [12]:
# >> Pipeline de Execucao

# 1. Download e definicao de caminhos
path = kagglehub.dataset_download("vinayakshanawad/heart-rate-prediction-to-monitor-stress-level")
caminhoDados = os.path.join(path, 'Train Data', 'Train Data Zip')

# 2. Ingestao e Unificacao
dfHrv = carregarDados(caminhoDados)

if dfHrv is not None:
    # 3. Selecao de Features e Mapeamento do Target (Binario)
    xBruto, yAlvo = selecionarPreditoresETarget(dfHrv)

    # 3.1 Aplicacao da Subamostragem
    xBalanceado, yBalanceado = aplicarSubamostragem(xBruto, yAlvo)

    # 4. Divisao do Conjunto de Dados (Estratificada)
    xTreinoBruto, xTesteBruto, yTreino, yTeste = dividirDados(
        xBalanceado, yBalanceado, proporcaoTeste=0.2
    )

    # 5. Transformacao (Normalidade via Yeo-Johnson + Padronizacao Z-score)
    xTreino, xTeste = aplicarTransformacoes(xTreinoBruto, xTesteBruto)

    print("\nPipeline concluido com sucesso!")


Dataset carregado! E unificado:
    (369289, 37)
Filtragem concluida.
  Preditores finais: 23 (Removidas 11 redundantes).
Subamostragem concluida: 128114 amostras totais (50/50).
Divisao dos conjuntos concluida (Teste: 20.0%).
Transformacao Yeo-Johnson e padronizacao z-score aplicadas.

Pipeline concluido com sucesso!


In [13]:
class MapeadorRBF:
    """
    Aproximacao do kernel RBF via Random Fourier Features.
    """

    def __init__(self, gamma=0.1, n_componentes=300, random_state=42):
        self.gamma = gamma
        self.n_componentes = n_componentes
        self.random_state = random_state
        self.W = None
        self.b = None

    def ajustar(self, xTreino):
        n_features = xTreino.shape[1]
        rng = np.random.default_rng(self.random_state)
        self.W = rng.normal(scale=np.sqrt(2 * self.gamma), size=(n_features, self.n_componentes))
        self.b = rng.uniform(0.0, 2 * np.pi, size=self.n_componentes)
        return self

    def transformar(self, xDados):
        z = xDados @ self.W + self.b
        return np.sqrt(2.0 / self.n_componentes) * np.cos(z)


In [14]:
class MeuClassificadorSVM:
    """
    Implementacao manual de SVM linear com margem suave (hinge loss).
    Otimizado via gradiente descendente com regularizacao L2.
    """

    def __init__(self, taxaAprendizado=0.001, epochs=2000, C=1.0, batch_size=256,
                 decay=0.001, patience=8, tol=1e-4, avaliar_intervalo=10):
        self.taxaAprendizado = taxaAprendizado
        self.epochs = epochs
        self.C = C
        self.batch_size = batch_size
        self.decay = decay
        self.patience = patience
        self.tol = tol
        self.avaliar_intervalo = avaliar_intervalo
        self.w = None
        self.b = 0.0

    def _preparar_rotulos(self, y):
        # Converte {0,1} -> {-1, +1}
        y = np.array(y)
        return np.where(y == 1, 1, -1)

    def _calcular_loss(self, xDados, yDados):
        margens = yDados * (np.dot(xDados, self.w) + self.b)
        hinge = np.maximum(0.0, 1.0 - margens)
        return 0.5 * np.dot(self.w, self.w) + self.C * np.mean(hinge)

    def treinarModelo(self, xTreino, yTreino):
        y = self._preparar_rotulos(yTreino)
        nAmostras, nPreditores = xTreino.shape
        self.w = np.zeros(nPreditores)
        self.b = 0.0

        best_w = None
        best_b = 0.0
        best_loss = float('inf')
        sem_melhora = 0

        # Treinamento por mini-batches
        for epoch in range(self.epochs):
            lr = self.taxaAprendizado / (1.0 + self.decay * epoch)
            indices = np.random.permutation(nAmostras)
            xShuffled = xTreino[indices]
            yShuffled = y[indices]

            for start in range(0, nAmostras, self.batch_size):
                end = start + self.batch_size
                xBatch = xShuffled[start:end]
                yBatch = yShuffled[start:end]

                margens = yBatch * (np.dot(xBatch, self.w) + self.b)
                viol = margens < 1

                # Gradiente do hinge loss + regularizacao L2
                grad_w = self.w - self.C * np.mean(yBatch[:, None] * xBatch * viol[:, None], axis=0)
                grad_b = -self.C * np.mean(yBatch * viol)

                self.w -= lr * grad_w
                self.b -= lr * grad_b

            if (epoch + 1) % self.avaliar_intervalo == 0:
                loss = self._calcular_loss(xTreino, y)
                if best_loss - loss > self.tol:
                    best_loss = loss
                    best_w = self.w.copy()
                    best_b = self.b
                    sem_melhora = 0
                else:
                    sem_melhora += 1
                    if sem_melhora >= self.patience:
                        if best_w is not None:
                            self.w = best_w
                            self.b = best_b
                        print(f"Early stopping em epoch {epoch + 1}.")
                        break

        print("Treino SVM concluido.")

    def predizer(self, xTeste):
        scores = np.dot(xTeste, self.w) + self.b
        return np.where(scores >= 0, 1, 0)


In [15]:
# 6. Busca de hiperparametros (gamma, n_componentes, C) com validacao
np.random.seed(42)
xTreinoBusca, xValBusca, yTreinoBusca, yValBusca = train_test_split(
    xTreino, yTreino, test_size=0.2, random_state=42, stratify=yTreino
)

gammas = [0.03, 0.05, 0.1, 0.2]
componentes = [300, 600]
Cs = [1.0, 2.0, 5.0]
epochs_busca = 800

best = {'acc': -1.0, 'gamma': None, 'n_componentes': None, 'C': None}

for gamma in gammas:
    for n in componentes:
        mapeador = MapeadorRBF(gamma=gamma, n_componentes=n, random_state=42)
        mapeador.ajustar(xTreinoBusca)
        xTreinoBuscaMap = mapeador.transformar(xTreinoBusca)
        xValBuscaMap = mapeador.transformar(xValBusca)

        for C in Cs:
            modelo = MeuClassificadorSVM(
                taxaAprendizado=0.001, epochs=epochs_busca, C=C, batch_size=256
            )
            modelo.treinarModelo(xTreinoBuscaMap, yTreinoBusca)
            yValPred = modelo.predizer(xValBuscaMap)
            acc = np.mean(yValPred == yValBusca)
            print(f"gamma={gamma} n={n} C={C} acc_val={acc:.2%}")

            if acc > best['acc']:
                best = {'acc': acc, 'gamma': gamma, 'n_componentes': n, 'C': C}

print(
    f"Melhor configuracao: gamma={best['gamma']} n={best['n_componentes']} C={best['C']} acc_val={best['acc']:.2%}"
)

# Treino final com o melhor conjunto
mapeador = MapeadorRBF(
    gamma=best['gamma'], n_componentes=best['n_componentes'], random_state=42
)
mapeador.ajustar(xTreino)
xTreinoMap = mapeador.transformar(xTreino)
xTesteMap = mapeador.transformar(xTeste)

modeloSvm = MeuClassificadorSVM(taxaAprendizado=0.001, epochs=2000, C=best['C'], batch_size=256)
modeloSvm.treinarModelo(xTreinoMap, yTreino)

yPreditoTreino = modeloSvm.predizer(xTreinoMap)
yPreditoTeste = modeloSvm.predizer(xTesteMap)


Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.03 n=300 C=1.0 acc_val=66.79%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.03 n=300 C=2.0 acc_val=66.76%
Early stopping em epoch 100.
Treino SVM concluido.
gamma=0.03 n=300 C=5.0 acc_val=66.79%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.03 n=600 C=1.0 acc_val=67.06%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.03 n=600 C=2.0 acc_val=67.05%
Early stopping em epoch 100.
Treino SVM concluido.
gamma=0.03 n=600 C=5.0 acc_val=67.04%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.05 n=300 C=1.0 acc_val=67.07%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.05 n=300 C=2.0 acc_val=67.13%
Early stopping em epoch 100.
Treino SVM concluido.
gamma=0.05 n=300 C=5.0 acc_val=67.06%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.05 n=600 C=1.0 acc_val=67.73%
Early stopping em epoch 90.
Treino SVM concluido.
gamma=0.05 n=600 C=2.0 acc_val=67.74%
Early stopping em epoch 100.


In [16]:
def calcularMetricas(yReal, yPrevisto):
    """
    Calcula metricas de desempenho para classificacao binaria e exibe
    um relatorio estruturado no console.
    """
    # 1. Calculos de Base (Matriz de Confusao)
    tp = np.sum((yReal == 1) & (yPrevisto == 1))
    tn = np.sum((yReal == 0) & (yPrevisto == 0))
    fp = np.sum((yReal == 0) & (yPrevisto == 1))
    fn = np.sum((yReal == 1) & (yPrevisto == 0))

    # 2. Calculo das Metricas Derivadas
    acuracia = (tp + tn) / (tp + tn + fp + fn)

    # Sensibilidade (Recall): Capacidade de detectar a classe 1 (Estresse)
    sensibilidade = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Especificidade: Capacidade de detectar a classe 0 (Calma)
    especificidade = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Precisao: Quando o modelo diz que e estresse, qual a chance de ser verdade?
    precisao = tp / (tp + fp) if (tp + fp) > 0 else 0

    # F1-Score: Media harmonica entre precisao e sensibilidade
    f1 = 2 * (precisao * sensibilidade) / (precisao + sensibilidade) if (precisao + sensibilidade) > 0 else 0

    # 3. Prints Organizados e Estruturados
    print("Relatorio de Desempenho da SVM")
    print("="*45)

    print(f"{'Metrica':<25} | {'Valor':<15}")
    print("-" * 45)
    print(f"{'Acuracia Global':<25} | {acuracia:.2%}")
    print(f"{'Sensibilidade (Recall)':<25} | {sensibilidade:.2%}")
    print(f"{'Especificidade':<25} | {especificidade:.2%}")
    print(f"{'Precisao':<25} | {precisao:.2%}")
    print(f"{'F1-Score':<25} | {f1:.4f}")

    print("\n\n")
    print("Matriz de Confusao")
    print("-" * 45)
    print(f"{'':>18} | {'Previsto: 0':<12} | {'Previsto: 1':<12}")
    print(f"{'Real: 0 (Calma)':>18} | {tn:<12} | {fp:<12}")
    print(f"{'Real: 1 (Estresse)':>18} | {fn:<12} | {tp:<12}")
    print("="*45 + "\n")

    matrizConfusao = np.array([[tn, fp], [fn, tp]])
    return acuracia, matrizConfusao


In [17]:
# 4. Calcular metricas finais no treino
print('Treino')
acuraciaGeralTreino, matrizConfusaoTreino = calcularMetricas(yTreino, yPreditoTreino)

# 4. Calcular metricas finais no teste
print('Teste')
acuraciaGeralTeste, matrizConfusaoTeste = calcularMetricas(yTeste, yPreditoTeste)


Treino
Relatorio de Desempenho da SVM
Metrica                   | Valor          
---------------------------------------------
Acuracia Global           | 83.45%
Sensibilidade (Recall)    | 86.22%
Especificidade            | 80.69%
Precisao                  | 81.70%
F1-Score                  | 0.8390



Matriz de Confusao
---------------------------------------------
                   | Previsto: 0  | Previsto: 1 
   Real: 0 (Calma) | 41349        | 9896        
Real: 1 (Estresse) | 7062         | 44184       

Teste
Relatorio de Desempenho da SVM
Metrica                   | Valor          
---------------------------------------------
Acuracia Global           | 83.51%
Sensibilidade (Recall)    | 86.15%
Especificidade            | 80.87%
Precisao                  | 81.83%
F1-Score                  | 0.8393



Matriz de Confusao
---------------------------------------------
                   | Previsto: 0  | Previsto: 1 
   Real: 0 (Calma) | 10361        | 2451        
Real: 1 (Estr